In [ ]:
%pip install requests pymysql python-dateutil cryptography

In [ ]:
import pymysql
from contextlib import contextmanager
from datetime import datetime
import time
import traceback
import requests

# Konfigurasi Database Aiven
DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "root"
DB_PASS = ""
DB_NAME = "aws_monitoring"

# URL API JSON
TARGET_URL = "http://202.90.199.132/aws-new/data/station/latest/3000000011"
SCRAPE_INTERVAL = 60  # detik (1 menit)


In [ ]:
@contextmanager
def get_conn():
    conn = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor,
        autocommit=True
    )
    try:
        yield conn
    finally:
        conn.close()

def ensure_table():
    sql = """
    CREATE TABLE IF NOT EXISTS aws_stat (
      id INT AUTO_INCREMENT PRIMARY KEY,
      idaws VARCHAR(50),
      waktu DATETIME,
      windspeed DOUBLE,
      winddir DOUBLE,
      temp DOUBLE,
      rh DOUBLE,
      pressure DOUBLE,
      rain DOUBLE,
      solrad DOUBLE,
      netrad DOUBLE,
      watertemp DOUBLE,
      waterlevel DOUBLE,
      ta_min DOUBLE,
      ta_max DOUBLE,
      pancilevel DOUBLE,
      pancitemp DOUBLE,
      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql)

ensure_table()
print("✅ Tabel sudah siap di MySQL Aiven")


✅ Tabel sudah siap di MySQL Aiven


In [ ]:
def insert_record(data):
    sql = """
    INSERT INTO aws_stat
    (idaws, waktu, windspeed, winddir, temp, rh, pressure, rain,
     solrad, netrad, watertemp, waterlevel, ta_min, ta_max, pancilevel, pancitemp)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    vals = (
        data.get("idaws"),
        data.get("waktu"),
        data.get("windspeed"),
        data.get("winddir"),
        data.get("temp"),
        data.get("rh"),
        data.get("pressure"),
        data.get("rain"),
        data.get("solrad"),
        data.get("netrad"),
        data.get("watertemp"),
        data.get("waterlevel"),
        data.get("ta_min"),
        data.get("ta_max"),
        data.get("pancilevel"),
        data.get("pancitemp")
    )
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, vals)

def fetch_json(url, timeout=15):
    resp = requests.get(url, timeout=timeout)
    resp.raise_for_status()
    return resp.json()


In [ ]:
print("🚀 Mulai scraping JSON ke MySQL Aiven... Tekan STOP (■) di Jupyter untuk hentikan")

while True:
    try:
        data = fetch_json(TARGET_URL)

        # Konversi waktu string ke datetime
        waktu = datetime.strptime(data["waktu"], "%Y-%m-%d %H:%M:%S")

        record = {
            "idaws": data.get("idaws"),
            "waktu": waktu,
            "windspeed": float(data.get("windspeed", 0)),
            "winddir": float(data.get("winddir", 0)),
            "temp": float(data.get("temp", 0)),
            "rh": float(data.get("rh", 0)),
            "pressure": float(data.get("pressure", 0)),
            "rain": float(data.get("rain", 0)),
            "solrad": float(data.get("solrad", 0)),
            "netrad": float(data.get("netrad", 0)),
            "watertemp": float(data.get("watertemp", 0)),
            "waterlevel": float(data.get("waterlevel", 0)),
            "ta_min": float(data.get("ta_min", 0)),
            "ta_max": float(data.get("ta_max", 0)),
            "pancilevel": float(data.get("pancilevel", 0)),
            "pancitemp": float(data.get("pancitemp", 0)),
        }

        insert_record(record)
        print(f"[{datetime.utcnow().isoformat()}] ✅ Data tersimpan ke Aiven: waktu={waktu}, temp={record['temp']}°C")

    except Exception as e:
        print("❌ Error:", str(e))
        traceback.print_exc()

    time.sleep(SCRAPE_INTERVAL)


🚀 Mulai scraping JSON ke MySQL Aiven... Tekan STOP (■) di Jupyter untuk hentikan


C:\Users\user\AppData\Local\Temp\ipykernel_12716\4092952038.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{datetime.utcnow().isoformat()}] ✅ Data tersimpan ke Aiven: waktu={waktu}, temp={record['temp']}°C")


[2025-10-11T07:26:57.049829] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:26:00, temp=33.1°C
[2025-10-11T07:27:57.364056] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:27:00, temp=33.1°C
[2025-10-11T07:28:57.675898] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:28:00, temp=33.1°C
[2025-10-11T07:29:57.890975] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:29:00, temp=33.2°C
[2025-10-11T07:30:58.185947] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:30:00, temp=33.1°C
[2025-10-11T07:31:58.356215] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:31:00, temp=33.1°C
[2025-10-11T07:32:58.488184] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:32:00, temp=33.1°C
[2025-10-11T07:33:58.678384] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:33:00, temp=33.0°C
[2025-10-11T07:34:58.893997] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:34:00, temp=33.1°C
[2025-10-11T07:35:59.145318] ✅ Data tersimpan ke Aiven: waktu=2025-10-11 07:35:00, temp=33.1°C
[2025-10-11T07:36:59.308158] ✅ Data tersimpan ke A